In [59]:
## Read Folder
## Loop annotation
## Read XML
## Extend BB
## Output XML

In [60]:
import glob
import os

In [61]:
from voc_io import *
## Read XML
def read_xml(path):
    return PascalVocReader(path)

def write_xml(outPath, info, bbs):
    '''
    outPath: output filepath
    info: PascalVocReader(path).getInfo()
    bbs: bounding boxes in foramt: ['name',xmin,xmax,ymin,ymax]
    '''
    writter = PascalVocWriter(info[0],info[1], (info[3],info[4], 3), localImgPath=info[2])
    for bb in bbs:
        writter.addBndBox(bb[1],bb[2],bb[3],bb[4], bb[0], difficult=0)
    writter.save(outPath)

In [62]:
def xml_processing(path, outPath, padding = None, ratio = None, default_voc_folder = "Sample_Anno_Data"):
    '''
    value: fix number of pixel
    ratio: resize the bouding box
    '''
    voc = read_xml(path)
    boxes = voc.getShapes()
    info = voc.getInfo()
    w,h = info[3],info[4]
    
    if info[0] is None:
        info[0] = default_voc_folder
    writter = PascalVocWriter(info[0],info[1], (w,h, 3), localImgPath=info[2])
    
    for box in boxes:
        xmin = int(box[1][0][0])
        xmax = int(box[1][1][0])
        ymin = int(box[1][0][1])
        ymax = int(box[1][2][1])
        
        xmin, xmax, ymin, ymax = applyPadding(xmin, xmax, ymin, ymax,padding, ratio, clipX=int(w), clipY=int(h))
        
        writter.addBndBox(xmin,ymin,xmax,ymax, box[0], 0)
        
    writter.save(outPath)
    
def applyPadding(xmin,xmax,ymin,ymax, padding=None, ratio=None, clipX=1000,clipY=1000):
    if ratio:
        x_diff = (xmax - xmin) * (ratio-1)
        y_diff = (ymax - ymin) * (ratio-1)
        xmin -= x_diff
        xmax += x_diff
        ymin -= y_diff
        ymax += y_diff

    if padding:
        xmin -= padding
        xmax += padding
        ymin -= padding
        ymax += padding

    xmin = int(max(0, xmin))
    xmax = int(min(int(clipX), xmax))
    ymin = int(max(0, ymin))
    ymax = int(min(int(clipY), ymax))
    
    return xmin,xmax,ymin,ymax


In [64]:
# Sample_Anno_Data_NPT, 15, 1.2
# Sample_Anno_Data_BYT  30, None
# Sample_Anno_Data_MTL, 10, 1.1


In [70]:
folder = "/Users/ckh/OneDrive - Default Directory/Hui_Wan/Sample_Anno_Data_BYT/annot"
outputFolder = "/Users/ckh/OneDrive - Default Directory/Hui_Wan/Sample_Anno_Data_BYT"

for files in os.scandir(folder):
    fileName = files.name
    xml = folder+"/"+ fileName
    outPath = outputFolder+"/"+fileName

#     xml_processing(xml, outPath, None, None)
    xml_processing(xml, outPath, 30, None)